In [31]:
def simulator():
    for file in os.listdir("./pickles"):
        if file.find("simulator"):
            os.remove("pickles/"+file)
    
    # choose a random model (matrix)
    Simulator_Model = namedtuple('Simulator_Model', ['id', 'T', 'D'])
    A = np.array(
                  [[ 9.99979469e-01,  2.74850285e-05,  1.80044529e-05],
                  [-3.69686852e-06,  1.00001277e+00, -5.63548129e-06],
                  [-1.15968968e-04,  2.40095928e-04,  1.00000545e+00]])
    B = np.array(
                    [[ 9.99977211e-01,  4.66229731e-05,  2.72804462e-06],
                    [-9.09166202e-06,  1.00001372e+00,  6.34163929e-06],
                    [ 1.01399198e-04, -2.21338930e-04,  1.00000168e+00]])
    
    C = np.array(
                    [[ 9.99965217e-01,  6.66674341e-05,  8.04332052e-06],
                     [-6.21151933e-06,  1.00002141e+00, -8.46187262e-06],
                     [-2.95084895e-04,  6.38211159e-04,  9.99992089e-01]])
    
    model1 = Simulator_Model(1, A, np.array([[0],[0],[0]]))
    model2 = Simulator_Model(2, B, np.array([[0],[0],[0]]))
    model3 = Simulator_Model(3, C, np.array([[0],[0],[0]]))
    model4 = Simulator_Model(4, matrix_power(A,4), np.array([[0],[0],[0]]))
    
    models = [model1, model4, model3]
    
    period_length_array = np.arange(25,35) # period_length/10
    
    # build the hr, bp, rr signals according to the model
    hr = []
    bp = []
    rr = []
    last_coloumn_data = np.array([[130],[65],[30]]) # hr[1]=130, bp[1]=65, rr[1]=30
    hr.append(last_coloumn_data[0])
    bp.append(last_coloumn_data[1])
    rr.append(last_coloumn_data[2])
    total_length = 0
    for j in range(3):
        model = random.choice(models)
        length = 10 * random.choice(period_length_array)
        total_length = total_length + length
        print("model " + str(model.id) + " from " + str(total_length - length) + " to " + str(total_length))
        
        for i in range(length * 1000):
            last_coloumn_data = np.dot(model.T, last_coloumn_data) + model.D
            hr.append(last_coloumn_data[0])
            bp.append(last_coloumn_data[1])
            rr.append(last_coloumn_data[2])
    data = np.row_stack((np.array(hr).transpose(),np.array(bp).transpose(),np.array(rr).transpose()))
    
    hr_noise = np.random.normal(0, 0.5, len(data[0]))
    bp_noise = np.random.normal(0, 0.5, len(data[0]))
    rr_noise = np.random.normal(0, 0.5, len(data[0]))
    
    hr = data[0] + hr_noise
    bp = data[1] + bp_noise
    rr = data[2] + rr_noise
    time = np.arange(0,len(hr)/1000,1/1000)

    plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
    plt.plot(time,hr)
    plt.title('Heart Rate')
    plt.xlabel('Time [sec]')
    plt.ylabel('[bpm]')
    plt.show()

    plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
    plt.plot(time,bp)
    plt.title('Systolic Blood Pressure')
    plt.xlabel('Time [sec]')
    plt.show()

    plt.figure(figsize=(FIG_WIDTH, FIG_HEIGHT))
    plt.plot(time,rr)
    plt.title('Respiratory Rate')
    plt.xlabel('Time [sec]')
    plt.show()

    
# Reverse the preproccesing process to get the original data set in csv files (ECG, blood pressure, resp)

    with open('samples/simulator-MDC_ECG_ELEC_POTL_II-500.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([0, 8300])
        last_peak = 0
        for j in np.arange(0.002,total_length-10, 0.002):
            i=round(j,3)
            if np.abs(60/(i-last_peak) - hr[(int)((last_peak+(i-last_peak)/2)/0.001)]) <= 2:
                writer.writerow([i, 8300])
                last_peak = i
            else:
                writer.writerow([i, 0])

    with open('samples/simulator-MDC_PRESS_BLD_ART_ABP-125.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        count = 0
        for j in np.arange(0,total_length-10, 0.008):
            i=round(j,3)
            if count%50 == 0:
                writer.writerow([i, ( bp[(int)(i/0.001)] + 40 ) / 0.0625])
            else:
                writer.writerow([i, 0])
            count = count + 1

    with open('samples/simulator-MDC_RESP-62.5.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([0, 1400])
        last_peak = 0
        for j in np.arange(0.016,total_length-10, 0.016):
            i=round(j,3)
            if np.abs(60/(i-last_peak) - rr[(int)((last_peak+(i-last_peak)/2)/0.001)]) <= 2:
                writer.writerow([i, 1400])
                last_peak = i
            else:
                writer.writerow([i, 0])
                